Установить библиотеки: pandas - для работы с таблицами, pyreadstat - для чтения файлов из статистических программ (типа SPSS). Обязательно указать версию библиотеки через ==, потому что код, работающий в одной версии, может не работать в другой.  
Документации на них:

https://pandas.pydata.org

https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html



In [1]:
!pip install pandas==1.5.3 pyreadstat==1.2.0 tqdm==4.65.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Подключить Google Disk, надо будет дать разрешение этой jupyter-notebook тетрадке на чтение/запись с него. Файлы *.sav должны быть заранее загружены на Google Disk в папку коуж. Если они загружены по другому пути, надо будет поменять переменную fpath (cм. ниже)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from collections import defaultdict

import pandas as pd
# импортируем библиотеку pyreadstat, 
# которая ориентирована на работу с форматами статистических (в т.ч., платных) программ.
import pyreadstat
from tqdm.notebook import tqdm as tqdm

In [4]:
fpath_diary_2019 = 'drive/MyDrive/использование_суточного_фонда_времени/BV_DIARYSVODO_2019.sav' # Источник данных: https://gks.ru/ .
df_diary_2019, meta_diary_2019 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_diary_2019, num_processes=16, encoding='cp1251') 
meta_diary_2019.column_names_to_labels

{'H00_02': 'Код субъекта РФ',
 'H00_04': 'Код типа населенного пункта',
 'H00_06': 'Номер домохозяйства в пределах территории',
 'H00_07': 'H00_07',
 'H01_00': 'Индивидуальный код члена домохозяйства',
 'H01_01': 'Пол респондента',
 'H01_02': 'Число исполнившихся лет',
 'D01': 'День недели, в течение которого заполняется дневник',
 'K_110': 'Работа в корпорациях, органах государственного управления и некоммерческих организациях',
 'K_121': 'Выращивание сельскохозяйственных культур на домашних предприятиях для реализации на рынке',
 'K_122': 'Разведение животных на домашних предприятиях для реализации на рынке',
 'K_123': 'Лесоводство и лесозаготовки на домашних предприятиях для реализации на рынке',
 'K_124': 'Ловля рыбы на домашних предприятиях для реализации на рынке',
 'K_125': 'Аквакультура на домашних предприятиях для реализации на рынке',
 'K_126': 'Подземная и открытая разработка полезных ископаемых на домашних предприятиях для реализации на рынке',
 'K_127': 'Производство и пер

In [5]:
meta_diary_2019.value_labels[meta_diary_2019.variable_to_label['R_1']]

{1.0: 'Городская местность', 2.0: 'Сельская местность'}

In [6]:
meta_diary_2019.value_labels[meta_diary_2019.variable_to_label['R_1_1']]

{1.0: 'городской, менее 50,0 тыс.чел.',
 2.0: 'городской,  50,0?99,9 тыс.чел.',
 3.0: 'городской,  100,0?249,9 тыс.чел.',
 4.0: 'городской,  250,0?499,9 тыс.чел.',
 5.0: 'городской,  500,0?999,9 тыс.чел.',
 6.0: 'городской,  1 млн. и более',
 7.0: 'сельский, 200 чел. и менее',
 8.0: 'сельский, 201?1000 чел.',
 9.0: 'сельский, 1001?5000 чел.',
 10.0: 'сельский, более 5000 чел.'}

In [7]:
selection = [
    'H00_02', # 'Код субъекта РФ',
    'H01_00', # 'Индивидуальный код члена домохозяйства',
    'D01', # 'День недели, в течение которого заполняется дневник',
    'K_721', # 'Участие в общественных торжествах, посвященных культурным/историческим событиям',
    'K_722', # 'Участие в общественных ритуалах/мероприятиях нерелигиозного характера, связанных со свадьбами, похоронами и рождениями, а также в аналоги?',
    'K_723', # '«Участие в общественных социальных мероприятиях (музыка, танцы и т.д.)
    'R_1', # 'Место проживания',
    'R_1_1', # 'Размер населенного пункта',
    'K_812', # 'Посещение парков/садов',
    'K_811', # 'Посещение организованных/массовых культурных мероприятий и выставок',
    'H00_06', # 'Номер домохозяйства в пределах территории',
    ] 

In [8]:
def select_variables(df, selection, meta):
    # Создадим новую таблицу (pandas DataFrame), но только с теми колонками,
    # которые используются в расчётах.
    # Так используется меньше памяти, удобнее визуализировать. Но можно пользоваться и исходной таблицей.
    selected = df[selection]
    # Используем генератор списков, чтобы переименовать служебные заголовки в читаемые.
    # для этого мы хотим "пройтись" по всем меткам в списке selection и получить новый список вместо selection.
    # Подробнее о генераторах списков: https://pythontutor.ru/lessons/lists/#section_3
    selection_names = [meta.column_names_to_labels[label].strip() for label in selection]
    # Заменим исходные наименования колонок на наименования из получившегося списка selection_names, 
    # теперь у нас наименования колонок будет как в столбце "метки" файла sav.
    selected.columns = selection_names
    return selected

In [9]:
whole_russia_2019 = select_variables(df_diary_2019, selection, meta_diary_2019)
whole_russia_2019.head(14)

,Код субъекта РФ,Индивидуальный код члена домохозяйства,"День недели, в течение которого заполняется дневник","Участие в общественных торжествах, посвященных культурным/историческим событиям","Участие в общественных ритуалах/мероприятиях нерелигиозного характера, связанных со свадьбами, похоронами и рождениями, а также в аналоги?","«Участие в общественных социальных мероприятиях (музыка, танцы и т.д.)»",Место проживания,Размер населенного пункта,Посещение парков/садов,Посещение организованных/массовых культурных мероприятий и выставок,Номер домохозяйства в пределах территории
0,01,1.0,6.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0
1,01,1.0,2.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0
2,01,1.0,3.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,2.0
3,01,1.0,6.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,2.0
4,01,1.0,4.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,3.0
5,01,1.0,7.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,3.0
6,01,1.0,5.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,4.0
7,01,1.0,7.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,4.0
8,01,1.0,2.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,5.0
9,01,1.0,6.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,5.0


Отличие этих данных от данных КОУЖ: для одного человека может быть более одной строки, так как один и тот же человек заполняет свой дневник в разные дни недели. Поэтому:

1. Группируем по кодам субъекта РФ
2. Группируем по домохозяйствам
3. Группируем по членам домохозяйства
4. Суммируем значения для всех дней недели по интересующим нас переменным
5. Теперь, когда для каждого человека одна строка, можно считать описательные статистики


In [10]:
# использовать только городскую местность
whole_russia_2019 = whole_russia_2019[whole_russia_2019['Место проживания']==1]

In [11]:
def select_region(df, code):
    mask = df['Код субъекта РФ'] == code
    # применяем маску - используем только те строчки, где значения маски True
    region = df[mask]
    # число наблюдений в таблице получаем через атрибут shape.
    # так как в shape находится объект типа кортеж (tuple)- структура данных, похожая на список,
    # но неизменяемая (чтобы добавить или удалить элемент, придется создавать новый объект типа кортеж в памяти).
    # из кортежа,  так же, как из списка, можно получить по порядковому индексу его элемент
    # в данном случае у нас shape[0] - количество строк, shape[1] - количество столбцов.
    # pandas DataFrame - двумерная таблица, ее размерность описывается количеством строчек и столбцов.
    # одномерная "таблица" pandas, по сути, одна колонка, размерность которой описывается только количеством строчек,
    # называется Series
    print(f"Количество наблюдений по региону {code}: {region.shape[0]}")
    return region

In [12]:
# проверка, какие регионы есть в данных
whole_russia_2019['Код субъекта РФ'].unique()

array(['01', '03', '04', '05', '07', '08', '10', '11', '12', '13', '14',
       '15', '17', '18', '19', '20', '22', '24', '25', '26', '27', '28',
       '29', '30', '32', '33', '34', '35', '36', '37', '38', '40', '41',
       '42', '44', '45', '46', '47', '49', '50', '52', '53', '54', '56',
       '57', '58', '60', '61', '63', '64', '65', '66', '67', '68', '69',
       '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80',
       '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91',
       '92', '93', '94', '95', '96', '97', '98', '99'], dtype=object)

In [13]:
my_region_2019 = select_region(whole_russia_2019, '98') # заменить номер региона на свой
my_region_2019_sums = defaultdict(list)
for household in my_region_2019['Номер домохозяйства в пределах территории'].unique():
    current_household = my_region_2019[my_region_2019['Номер домохозяйства в пределах территории'] == household]
    for member in current_household['Индивидуальный код члена домохозяйства'].unique():
        current_member = current_household[current_household['Индивидуальный код члена домохозяйства'] == member]
        my_region_2019_sums['Размер населенного пункта'].append(current_member['Размер населенного пункта'].iloc[0])
        my_region_2019_sums['Участие в общественных торжествах, посвященных культурным/историческим событиям'].append(current_member['Участие в общественных торжествах, посвященных культурным/историческим событиям'].sum())
        my_region_2019_sums['Участие в общественных ритуалах/мероприятиях нерелигиозного характера, связанных со свадьбами, похоронами и рождениями, а также в аналоги?'].append(current_member['Участие в общественных ритуалах/мероприятиях нерелигиозного характера, связанных со свадьбами, похоронами и рождениями, а также в аналоги?'].sum())
        my_region_2019_sums['«Участие в общественных социальных мероприятиях (музыка, танцы и т.д.)»'].append(current_member['«Участие в общественных социальных мероприятиях (музыка, танцы и т.д.)»'].sum())
        my_region_2019_sums['Посещение парков/садов'].append(current_member['Посещение парков/садов'].sum())
        my_region_2019_sums['Посещение организованных/массовых культурных мероприятий и выставок'].append(current_member['Посещение организованных/массовых культурных мероприятий и выставок'].sum())
my_region_2019_sums = pd.DataFrame(my_region_2019_sums)
my_region_2019_sums.head()

Количество наблюдений по региону 98: 755


,Размер населенного пункта,"Участие в общественных торжествах, посвященных культурным/историческим событиям","Участие в общественных ритуалах/мероприятиях нерелигиозного характера, связанных со свадьбами, похоронами и рождениями, а также в аналоги?","«Участие в общественных социальных мероприятиях (музыка, танцы и т.д.)»",Посещение парков/садов,Посещение организованных/массовых культурных мероприятий и выставок
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


### Описательные статистики для городов разного размера данного региона

In [14]:
for size in my_region_2019_sums['Размер населенного пункта'].unique():
    print(meta_diary_2019.value_labels[meta_diary_2019.variable_to_label['R_1_1']][size])
    my_region_2019_sums_by_size = my_region_2019_sums[my_region_2019_sums['Размер населенного пункта']==size]
    for variable in my_region_2019_sums_by_size.columns[1:]:
        print(variable)
        print(my_region_2019_sums_by_size[variable].describe())
    print('----------------------------------------------------------------------------------')

городской, менее 50,0 тыс.чел.
Участие в общественных торжествах, посвященных культурным/историческим событиям
count    235.000000
mean       0.468085
std        7.175609
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max      110.000000
Name: Участие в общественных торжествах, посвященных культурным/историческим событиям, dtype: float64
Участие в общественных ритуалах/мероприятиях нерелигиозного характера, связанных со свадьбами, похоронами и рождениями, а также в аналоги?
count    235.000000
mean       4.978723
std       43.876034
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max      390.000000
Name: Участие в общественных ритуалах/мероприятиях нерелигиозного характера, связанных со свадьбами, похоронами и рождениями, а также в аналоги?, dtype: float64
«Участие в общественных социальных мероприятиях (музыка, танцы и т.д.)»
count    235.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%  

## Всё то же самое, но для 2014 года

In [15]:
fpath_2014 = 'drive/MyDrive/использование_суточного_фонда_времени/BV_Diary_public_2014.sav'
df_2014, meta_2014 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2014, num_processes=16, encoding='cp1251') 
meta_2014.column_names_to_labels

{'HH_number': 'Номер домохозяйства (сквозной)',
 'D00_08': 'Индивидуальный код члена домохозяйства',
 'BD': 'Будний день',
 'RD': 'Рабочий день',
 'PVD': 'Предвыходной день',
 'VD': 'Выходной день',
 'K_10': 'Рабочее время на основной работе в формальном секторе',
 'K_20': 'Рабочее время на дополнительной работе в формальном секторе',
 'K_30': 'Рабочее время в качестве ученика, стажера, подмастерья в формальном секторе',
 'K_40': 'Короткие перерывы в работе в формальном секторе',
 'K_50': 'Обучение,  профподготовка/переподготовка  в связи с работой в «формальном секторе»',
 'K_80': 'Деятельность, связанная с поиском работы (для желающих работать в формальном секторе)',
 'K_90': 'Организация работы предприятия, деятельность по созданию предприятия в формальном секторе',
 'K_100': 'Передвижения, связанные с работой в «формальном секторе»',
 'K_110': 'Работав «формальномсекторе», неотнесеннаякдругимкатегориям',
 'K_121': 'Освоение земельного участка (основная работа)',
 'K_122': 'Освоение

In [16]:
meta_2014.value_labels[meta_2014.variable_to_label['R_1_1']]

{1.0: 'менее 50,0 тыс. человек',
 2.0: '50,0-99,9 тыс. человек',
 3.0: '100,0-249,9 тыс. человек',
 4.0: '250,0-499,9 тыс. человек',
 5.0: '5500,0-499,9 тыс. человек',
 6.0: '1 млн.человек и более',
 7.0: 'до 200 человек',
 8.0: '201-1000 человек',
 9.0: '1001-5000 человек',
 10.0: 'более 5000 человек'}

In [17]:
meta_2014.value_labels[meta_2014.variable_to_label['R_1']]

{1.0: 'городская местность', 2.0: 'сельская местность'}

In [18]:
selection_2014 = [ # набор переменных в 2014 году отличается от 2019
    'D00_08', # 'Индивидуальный код члена домохозяйства',
    'BD', #'Будний день',
    'RD', #'Рабочий день',
    'PVD', #'Предвыходной день',
    'VD', # 'Выходной день',
    'K_1870', # 'Участие в торжествах, посвященных культурным/историческим событиям',
    'K_1880', # 'Участие в обрядах/мероприятияхнерелигиозного характера',
    'K_1890', # 'Участие в общественных социальных мероприятиях',
    'R_1', # 'Место проживания',
    'R_1_1', # 'Размер населенного пункта',
    'K_1930', # 'Посещение музеев, художественных галерей, исторических парков/парков культуры, памятников старины',
    'HH_number', #'Номер домохозяйства (сквозной)'
    ] 
whole_russia_2014 = select_variables(df_2014, selection_2014, meta_2014)
whole_russia_2014.head()

,Индивидуальный код члена домохозяйства,Будний день,Рабочий день,Предвыходной день,Выходной день,"Участие в торжествах, посвященных культурным/историческим событиям",Участие в обрядах/мероприятияхнерелигиозного характера,Участие в общественных социальных мероприятиях,Место проживания,Размер населенного пункта,"Посещение музеев, художественных галерей, исторических парков/парков культуры, памятников старины",Номер домохозяйства (сквозной)
0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,2.0
3,1.0,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0
4,2.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,2.0


In [19]:
# в данных за 2014 нет кода субъекта РФ
sums = defaultdict(list)
# домохозяйств много, а у нас вложенный цикл (вычислительная сложность О(n^^2) ), поэтому придется подождать полминуты
# чтобы ждать было не скучно, добавим красивую бегущую строчку, обернув переменную, по которой итерируемся, в tqdm
for household in tqdm(whole_russia_2014['Номер домохозяйства (сквозной)'].unique()): 
    current_household = whole_russia_2014[whole_russia_2014['Номер домохозяйства (сквозной)'] == household]
    for member in current_household['Индивидуальный код члена домохозяйства'].unique():
        current_member = current_household[current_household['Индивидуальный код члена домохозяйства'] == member]
        sums['Размер населенного пункта'].append(current_member['Размер населенного пункта'].iloc[0])
        sums['Участие в торжествах, посвященных культурным/историческим событиям'].append(current_member['Участие в торжествах, посвященных культурным/историческим событиям'].sum())
        sums['Участие в обрядах/мероприятияхнерелигиозного характера'].append(current_member['Участие в обрядах/мероприятияхнерелигиозного характера'].sum())
        sums['Участие в общественных социальных мероприятиях'].append(current_member['Участие в общественных социальных мероприятиях'].sum())
        sums['Посещение музеев, художественных галерей, исторических парков/парков культуры, памятников старины'].append(current_member['Посещение музеев, художественных галерей, исторических парков/парков культуры, памятников старины'].sum())
sums = pd.DataFrame(sums)
for size in sums['Размер населенного пункта'].unique():
    print(meta_2014.value_labels[meta_2014.variable_to_label['R_1_1']][size])
    sums_by_size = sums[sums['Размер населенного пункта']==size]
    for variable in sums_by_size.columns[1:]:
        print(variable)
        print(sums_by_size[variable].describe())
    print('----------------------------------------------------------------------------------')

  0%|          | 0/9996 [00:00<?, ?it/s]

менее 50,0 тыс. человек
Участие в торжествах, посвященных культурным/историческим событиям
count    2829.000000
mean        0.109579
std         4.372427
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       210.000000
Name: Участие в торжествах, посвященных культурным/историческим событиям, dtype: float64
Участие в обрядах/мероприятияхнерелигиозного характера
count    2829.000000
mean        4.079180
std        39.113796
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       760.000000
Name: Участие в обрядах/мероприятияхнерелигиозного характера, dtype: float64
Участие в общественных социальных мероприятиях
count    2829.000000
mean        0.643337
std        12.385674
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       400.000000
Name: Участие в общественных социальных мероприятиях, dtype: float64
Посещение музеев, художественных галерей, исторических парков/парков